# Factors Driving Austin Housing Prices
## Build POI Index
Ricardo Dunia (duniarh)
<br/>
Kevin John Cherian Joseph (kc43529)
<br/>
Arnob Mallick (am93746)

In [21]:
import pandas as pd
import numpy as np
import json
import requests
import time
import csv

In [22]:
# foursquare config and secrets
clientId = 'your-client-id'
clientSecret = 'your-client-secret'
url = 'https://api.foursquare.com/v2/venues/search'

In [23]:
# define parameters for austin grid segmentation
segLat = 50
segLon = 50
minLat = 30.0727239
maxLat = 30.5193782
minLon = -98.0158212
maxLon = -97.4053586
delLat = (maxLat - minLat) / segLat
delLon = (maxLon - minLon) / segLon

In [27]:
# function to generate a generic property index
def generateIndex(categories, query, radius, byCategory=True):
    genIndex = [[ 0 for x in range(segLon)] for y in range(segLat)]

    # track if api requests have been limited
    rateLimited = False

    # iterate through segments
    for lon in range(segLon):
        if rateLimited:
            break
            
        for lat in range(segLat):
            if rateLimited:
                break

            searchLat = minLat + (delLat * (lat + 0.5))
            searchLon = minLon + (delLon * (lon + 0.5))

            # build query parameters
            params = dict(
                client_id=clientId,
                client_secret=clientSecret,
                ll=(str(searchLat) + ',' + str(searchLon)),
                intent='browse',
                v='20180323',
                radius=radius,
                limit=50
            )
            if byCategory:
                params['categoryId'] = categories
            else:
                params['query'] = query

            # make the request to foursquare
            resp = requests.get(url=url, params=params)
            if (resp.status_code != 200):
                time.sleep(0.1)
                resp = requests.get(url=url, params=params)
            if (resp.status_code != 200):
                time.sleep(0.1)
                resp = requests.get(url=url, params=params)
            if (resp.status_code != 200):
                time.sleep(0.1)
                resp = requests.get(url=url, params=params)

            data = json.loads(resp.text)
            if data['response']:
                numLocs = len(data['response']['venues'])
                genIndex[lat][lon] = numLocs 
                print('\rbuilt: {} with {} locations'.format(str(lat) + 'x' + str(lon), numLocs), end="\r")
            else:
                print('api rate limit exceeded')
                print(data)
                rateLimited = True
            
            # sleep before moving to the next block
            time.sleep(0.5)

    return genIndex

In [29]:
# define feature
targetQuery = ''
targetCategoryIds = ''

# generate and save output
genIndex = generateIndex(targetCategoryIds, targetQuery, 1500, False)
with open('./data/poi_index_' + targetQuery.replace(' ', '_') + '.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(genIndex)

built: 49x49 with 0 locations